This notebook is a follow-along from this Medium post: 
    https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b
    
Some of the code doesn't work correctly at the moment and I've made minor modifications, but the bulk works well enough to get started

In [5]:
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import time

In [6]:
URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup.prettify())

In [7]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

['Data Scientist',
 'Data Scientist',
 'Data Scientist - Intermediate',
 'Data Scientist',
 'Data Scientist, Music Analytics',
 'Associate Data Scientist, Ads',
 'Data Scientist',
 'Data Scientist, Mid',
 'Data Scientist',
 'Staff Data Scientist- Risk',
 'Data Scientist (Personalization)',
 'People Data Scientist, People Analytics',
 'Data Scientist',
 'Data Scientist, Personalization',
 'Data Scientist']

In [8]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['Altice',
 'MassMutual',
 'Envision LLC',
 'IBM',
 'Spotify',
 'Spotify',
 'QuaEra',
 'Booz Allen Hamilton',
 'INFICON, Inc.',
 'States Title',
 'The New York Times',
 'Uber',
 'Amplify Education, Inc.',
 'Hungryroot',
 'FanDuel']

In [9]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

['Long Island City, NY',
 'New York, NY 10004 (Financial District area)',
 'Buffalo, NY 14201 (Lakeview area)',
 'New York, NY',
 'New York, NY',
 'New York, NY 10018 (Garment District area)',
 'Rome, NY 13441',
 'East Syracuse, NY 13057',
 'New York State',
 'New York, NY',
 'New York, NY',
 'Brooklyn, NY',
 'New York State',
 'New York, NY']

In [10]:
# this part doesn't seem to work correctly
def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name='div', attrs={'class':'sjcl'})
                div_three = div_two.find('div')
                salaries.append(div_three.text.strip())
            except:
                salaries.append('Nothing_found')
    return(salaries)
extract_salary_from_result(soup)

['Altice\n\n\n\n3.4',
 'MassMutual\n\n\n\n3.7',
 'Envision LLC\n\n\n\n3.3',
 'IBM\n\n\n\n3.9',
 'Spotify\n\n\n\n4.3',
 'Spotify\n\n\n\n4.3',
 'QuaEra',
 'Booz Allen Hamilton\n\n\n\n3.9',
 'INFICON, Inc.',
 'States Title\n\n\n\n3.2',
 'The New York Times\n\n\n\n4.0',
 'Uber\n\n\n\n3.7',
 'Amplify Education, Inc.\n\n\n\n3.9',
 'Hungryroot\n\n\n\n3.5',
 'FanDuel\n\n\n\n4.1']

In [11]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

[]

In [12]:
max_results_per_city = 100
city_set = ['Chicago', 'Houston', 'Miami']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']
sample_df = pd.DataFrame(columns = columns)

In [13]:
sample_df

,city,job_title,company_name,location,summary,salary


In [14]:
new_array = []
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, 'lxml', from_encoding='utf-8')
        for div in soup.find_all(name='div', attrs={'class':'row'}): 
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city) 
            #grabbing job title
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                job_post.append(a['title']) 
            #grabbing company name
            company = div.find_all(name='span', attrs={'class':'company'}) 
            if len(company) > 0: 
                for b in company:
                    job_post.append(b.text.strip()) 
            else: 
                sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                for span in sec_try:
                    job_post.append(span.text) 
            #grabbing location name
            c = div.findAll('span', attrs={'class': 'location'}) 
            for span in c: 
                job_post.append(span.text) 
            #grabbing summary text
            d = div.findAll('span', attrs={'class': 'summary'}) 
            for span in d:
                job_post.append(span.text.strip()) 
            #grabbing salary
            try:
                job_post.append(div.find('nobr').text) 
            except:
                try:
                    div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                    div_three = div_two.find('div') 
                    job_post.append(div_three.text.strip())
                except:
                    job_post.append('Nothing_found') 
            #appending list of job post info to dataframe at index num
            new_array.append(job_post)


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [15]:
pd.DataFrame(new_array,columns=sample_df.columns[:-1])

,city,job_title,company_name,location,summary
0,Chicago,Data Scientist,North American Corporation,"Glenview, IL",North American Corporation\n\n\n\n3.1
1,Chicago,Senior Data Scientist - BAM Ventures,Balyasny,"Chicago, IL 60602 (The Loop area)",Balyasny\n\n\n\n4.0
2,Chicago,Senior Data Scientist - Enterprise Data Science,"Blue Cross Blue Shield of IL, MT, NM, OK & TX","Chicago, IL","Blue Cross Blue Shield of IL, MT, NM, OK & TX\..."
3,Chicago,Sr.Data Scientist,Sky Consulting Inc,"Chicago, IL",Sky Consulting Inc
4,Chicago,Data Scientist,Underwriters Laboratories Inc.,"Northbrook, IL 60062",Underwriters Laboratories Inc.\n\n\n\n3.6
...,...,...,...,...,...
197,Miami,AWS Data Engineer,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
198,Miami,Data Science Consultant,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
199,Miami,Data Governance and Quality Analytics & Cognit...,Deloitte,"Miami, FL 33131 (Downtown area)",Deloitte\n\n\n\n4.0
200,Miami,Google Cloud Data Engineer,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
